<a href="https://colab.research.google.com/github/wuchienhsun/Spaceship-Titanic/blob/main/Spaceship_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import numpy as np
import string

In [71]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [72]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [73]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [74]:
# Port => P => left and starboard => S => right
# separate Cabin => deck-num/side
# isSame group check if is family using passengerId and Name to know

In [75]:
# split cabin function
letter_count = dict(zip(string.ascii_uppercase, range(1,27)))

group_dict = {}

def split_cabins(cabins):
  cp_list = []
  side_list = []
  grouping_list = []
  for cabin in cabins:
    cp, side = split_cabin(cabin)
    cp_list.append(cp)
    side_list.append(side)
  print('group_dict', group_dict)
  for cabin in cabins:
    grouping_list.append(check_is_grouping(cabin)) 
  return cp_list, side_list, grouping_list

def split_cabin(cabin):
  cabin_data_list = cabin.split('/')
  group_key = cabin_data_list[0] + cabin_data_list[1]
  if group_key in group_dict:
    group_dict[group_key] += 1
  else:
    group_dict[group_key] = 1

  cabin_position =  letter_count[cabin_data_list[0]]
  if cabin_data_list[2] == 'S':
    # S
    side = 1
  else:
    # P
    side = 0
  return cabin_position, side

def check_is_grouping(cabin):
  cabin_data_list = cabin.split('/')
  group_key = cabin_data_list[0] + cabin_data_list[1]
  if group_key in group_dict and group_dict[group_key] > 1:
    return 1
  else:
    return 0

In [77]:
df_test = test
df_train = train
df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test.dropna(inplace = True)
df_train.replace([np.inf, -np.inf], np.nan, inplace = True)
df_train.dropna(inplace = True)
full_data = [df_train, df_test]

for dataset in full_data:
  dataset['HomePlanet'] = dataset['HomePlanet'].map( {'Earth': 0, 'Europa': 1, 'Mars': 2} ).astype(int, errors='ignore')
  dataset['CryoSleep'] = dataset['CryoSleep'].map( {False: 0, True: 1} ).astype(int, errors='ignore')
  dataset['Destination'] = dataset['Destination'].map( {'55 Cancri e': 0, 'PSO J318.5-22': 1, 'TRAPPIST-1e': 2} ).astype(int, errors='ignore')
  # dataset['Age']
  dataset['VIP'] = dataset['VIP'].map( {False: 0, True: 1} ).astype(int, errors='ignore')
  cp, side, grouping = split_cabins(dataset['Cabin'])
  dataset['CabinPosition'] = cp
  dataset['Side'] = side
  dataset['Grouping'] = grouping
  if 'Transported' in dataset:
    dataset['Transported'] = dataset['Transported'].map( {False: 0, True: 1} ).astype(int, errors='ignore')

group_dict {'B0': 3, 'F0': 4, 'A0': 6, 'F1': 4, 'F2': 4, 'F3': 2, 'B1': 6, 'G1': 2, 'F4': 3, 'F5': 3, 'G0': 2, 'F6': 4, 'E0': 9, 'C0': 3, 'F8': 2, 'G4': 3, 'F9': 5, 'D1': 4, 'F10': 4, 'G2': 2, 'B3': 6, 'G3': 7, 'E2': 3, 'F11': 3, 'A1': 7, 'G7': 3, 'F12': 3, 'F13': 4, 'G6': 2, 'G10': 5, 'F15': 4, 'F14': 2, 'F17': 4, 'D3': 4, 'C3': 4, 'F18': 3, 'C4': 5, 'G13': 3, 'F16': 4, 'G14': 2, 'C5': 4, 'E5': 2, 'G15': 3, 'F20': 2, 'G9': 5, 'A2': 2, 'G11': 5, 'F19': 3, 'F23': 4, 'G18': 6, 'D2': 3, 'G19': 4, 'B5': 6, 'E6': 3, 'G20': 4, 'D4': 3, 'G21': 3, 'F27': 4, 'E7': 3, 'E8': 4, 'F29': 4, 'D5': 4, 'G17': 2, 'C6': 6, 'F31': 4, 'G25': 6, 'G26': 8, 'G22': 2, 'B8': 4, 'G28': 4, 'F35': 4, 'G23': 2, 'E10': 2, 'G30': 2, 'G24': 3, 'F38': 3, 'B2': 2, 'D6': 2, 'G27': 2, 'F36': 3, 'G31': 4, 'G29': 7, 'F41': 8, 'F42': 4, 'G33': 2, 'B9': 2, 'C8': 3, 'G34': 3, 'G32': 2, 'F44': 8, 'E9': 3, 'F45': 4, 'F46': 2, 'F40': 3, 'F47': 2, 'G36': 2, 'G37': 12, 'F48': 3, 'E13': 15, 'E14': 3, 'G35': 3, 'G38': 2, 'F43': 3, 'B